In [57]:
import numpy as np
import itertools
import heapq

class QuoridorState:
    '''
    Quoridor state container class - generalized by size N
    Attributes:
        positions: 2x2 tuple representing the positions of the players
        left_wall: 1x2 tuple representing the number of walls left for each player
        walls: NxNx2 numpy array representing the walls
    '''
    def __init__(self, N: int = 9, n_walls: int = 10, copy: 'QuoridorState' = None):
        self.N = N
        if copy is not None:
            self.N = copy.N
            self.positions = copy.positions.copy()
            self.left_wall = copy.left_wall.copy()
            self.walls = copy.walls.copy()
            self.board = copy.board.copy()
        else:
            self.positions = np.array([[0, self.N // 2], [self.N - 1, self.N // 2]])
            self.left_wall = np.array([n_walls, n_walls])
            self.walls = np.zeros((2, self.N - 1, self.N - 1), dtype=np.int8)
            self.board = self.init_board()


    def init_board(self):
        '''
        Returns a 2N-1x2N-1 numpy array representing the board
        '''
        board = np.zeros((self.N * 2 - 1, self.N * 2 - 1), dtype=np.int8)
        board[1::2, 1::2] = 1
        board[self.positions[0, 0] * 2, self.positions[0, 1] * 2] = 2
        board[self.positions[1, 0] * 2, self.positions[1, 1] * 2] = 3
        return board

    def copy(self):
        return QuoridorState(copy=self)



class Quoridor:
    '''
    Quoridor rule management class
    '''
    def __init__(self, N, n_walls):
        self.N = N
        self.n_walls = n_walls
    
    def get_initial_state(self):
        '''
        Returns the initial state of the game
        '''
        state = QuoridorState(self.N, self.n_walls)
        return state
    
    def _search_on_board(self, state: QuoridorState, player):
        '''
        Returns the distance of shortest path to the goal of given player using a* algorithm.
        1 is wall, 0 is path
        if player 0: end at (2N - 2, *)
        if player 1: end at (0, *)
        heuristic: column-distance to the goal
        '''
        board = state.board
        now_pos = state.positions[player] * 2
        queue = []
        heuristic = lambda pos: (2 * self.N - 2 - pos[0]) * (1 - player) + (2 * pos[0]) * player
        heapq.heappush(queue, (-heuristic(now_pos), 0, -1, now_pos))
        visited = np.zeros((2 * self.N - 1, 2 * self.N - 1), dtype=np.int8)
        for cnt in itertools.count():
            if len(queue) == 0:
                return -1
            _, g, _, pos = queue.pop()
            if pos[0] < 0 or pos[0] > 2 * self.N - 2 or pos[1] < 0 or pos[1] > 2 * self.N - 2:
                continue
            if board[*pos] == 1 or visited[*pos] == 1:
                continue
            if pos[0] == (2 * self.N - 2) * (1 - player):
                return g
            visited[*pos] = 1
            for i in range(4):
                e = np.array([[-1, 0], [1, 0], [0, -1], [0, 1]])[i]
                heapq.heappush(queue, (-(heuristic(pos + e) + g + 1), g + 1, 4 * cnt + i, pos + e))


    def is_valid_wall(self, state: QuoridorState):
        '''
        Returns True if the state is valid, False otherwise
        Conditions for a valid state:
            - Walls are not blocking the path to the goal
        '''
        for i in range(2):
            if self._search_on_board(state, i) == -1:
                return False
        return True
    
    def _search_valid_moves(self, state: QuoridorState, player):
        '''
        Returns a list of valid moves from the given position using dfs
        1 is wall, 0 is path
        2 is player 0, 3 is player 1
        stack: [(pos, n_step)]
        n_step stops at 2
        reset step when board[pos] == 2 or 3
        '''
        board = state.board
        now_pos = state.positions[player] * 2
        movable = []
        stack = []
        stack.append((now_pos, 0))
        visited = np.zeros((2 * self.N - 1, 2 * self.N - 1), dtype=np.int8)
        while len(stack) > 0:
            pos, step = stack.pop()
            if pos[0] < 0 or pos[0] > 2 * self.N - 2 or pos[1] < 0 or pos[1] > 2 * self.N - 2:
                continue
            if board[*pos] == 1 or visited[*pos] == 1:
                continue
            if board[*pos] == 2 or board[*pos] == 3:
                step = 0
            visited[*pos] = 1
            if step == 2:
                movable.append(pos // 2)
                continue
            for e in np.array([[-1, 0], [1, 0], [0, -1], [0, 1]]):
                stack.append((pos + e, step + 1))

        return movable


    def is_valid_move(self, state: QuoridorState, next_pos, player):
        '''
        Returns True if the next state is a valid move, False otherwise
        Conditions for a valid move:
            - The player is moving to a valid position
        '''
        cvt_pos = np.array(next_pos)
        if any([np.array_equal(cvt_pos, val_mov) for val_mov in self._search_valid_moves(state, player)]):
            return True
        else:
            return False


    def get_next_state(self, state: QuoridorState, action: tuple, player: int):
        '''
        Returns the next state of the game given the current state and action
        '''
        action_type, action_value = action
        next_state = QuoridorState(copy=state)
        if action_type == 0:
            if self.is_valid_move(next_state, action_value, player):
                next_state.board[*next_state.positions[player] * 2] = 0
                next_state.board[*np.array(action_value) * 2] = player + 2
                next_state.positions[player] = action_value
                return next_state
            else:
                # print('Invalid move')
                return None
        else:
            if next_state.left_wall[player] == 0:
                # print('No wall left')
                return None
            
            hv, row, col = action_value
            if next_state.walls[hv, row, col] != 0:
                # print('Invalid wall')
                return None
            
            next_state.walls[hv, row, col] = 1
            next_state.walls[1 - hv, row, col] = -1
            if hv == 0 and col > 0:
                next_state.walls[0, row, col - 1] = -1
            if hv == 1 and row > 0:
                next_state.walls[1, row - 1, col] = -1
            if hv == 0 and col < self.N - 2:
                next_state.walls[0, row, col + 1] = -1
            if hv == 1 and row < self.N - 2:
                next_state.walls[1, row + 1, col] = -1
            next_state.board[
                row * 2 - hv + 1 : row * 2 + hv + 2,
                col * 2 - (1 - hv) + 1 : col * 2 + (1 - hv) + 2
            ] = 1
            next_state.left_wall[player] -= 1
            if not self.is_valid_wall(next_state):
                # print('Invalid wall')
                return None

            return next_state

    def get_valid_actions(self, state: QuoridorState, player: int):
        moves = self._search_valid_moves(state, player)
        walls = [
            (a, b, c) 
            for a in range(2) for b in range(self.N - 1) for c in range(self.N - 1) 
            if self.get_next_state(state, (1, (a, b, c)), player) is not None
        ]
        return [(0, tuple(move)) for move in moves] + [(1, tuple(wall)) for wall in walls]

    def check_win(self, state: QuoridorState, player):
        '''
        Returns True if the player wins, False otherwise
        '''
        if state.positions[player][0] == (self.N - 1) * (1 - player):
            return True
        else:
            return False
    
    def get_draw_value(self, state: QuoridorState, player: int):
        '''
        Returns the reward of the given state. Possibly value can be heuristic, not only win-lose.
        '''
        p_value = self._search_on_board(state, player)
        o_value = self._search_on_board(state, 1 - player)
        return o_value / (p_value + o_value)
        
    def get_value_and_terminated(self, state: QuoridorState, player: int, turn_count: int):
        '''
        Returns whether the game is terminated and the reward of the given state.
        If the game progresses more than 50 turns, the game is forced to terminate.
        '''
        if turn_count > 50:
            return True, True, self.get_draw_value(state, player)
        if self.check_win(state, player):
            return True, False, 1
        else:
            return False, False, 0
        

def parse_cmd(cmd: str) -> tuple:
    s = cmd.split(' ')
    if s[0] == 'move':
        return 0, (int(s[1]), int(s[2]))
    elif s[0] == 'wall':
        return 1, (int(s[1]), int(s[2]), int(s[3]))
    else:
        raise ValueError('Invalid action type')


In [59]:
test_quoridor = Quoridor(3, 1)
test_state = test_quoridor.get_initial_state()
test_state = test_quoridor.get_next_state(test_state, (1, (0, 1, 1)), 0)
print(test_state.board, test_state.left_wall)

test_state = test_quoridor.get_next_state(test_state, (1, (0, 0, 0)), 1)
test_state = test_quoridor.get_next_state(test_state, (0, (0, 0)), 0)
print(test_state.board, test_state.left_wall)
print(test_quoridor.get_draw_value(test_state, 0))
print(test_quoridor.get_draw_value(test_state, 1))


[[0 0 2 0 0]
 [0 1 0 1 0]
 [0 0 0 0 0]
 [0 1 1 1 1]
 [0 0 3 0 0]] [0 1]
[[2 0 0 0 0]
 [1 1 1 1 0]
 [0 0 0 0 0]
 [0 1 1 1 1]
 [0 0 3 0 0]] [0 0]
0.45454545454545453
0.5454545454545454


In [58]:
import math
import random

import tqdm


class Node:
    def __init__(
            self,
            game: Quoridor,
            args,
            state: QuoridorState,
            player,
            turn_count: int,
            parent: 'Node' = None,
            action_taken = None):
        
        self.game = game
        self.args = args
        self.state = state
        self.player = player
        self.turn_count = turn_count
        self.parent = parent
        self.action_taken = action_taken

        self.children: list['Node'] = []
        self.expandable_actions = self.game.get_valid_actions(self.state, self.player)

        self.visit_count = 0
        self.total_value = 0

    def is_fully_expanded(self):
        return len(self.expandable_actions) == 0 and len(self.children) > 0
    
    def select(self) -> 'Node':
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_ucb = ucb
                best_child = child

        return best_child

    def get_ucb(self, child: 'Node'):
        q_value = child.total_value / child.visit_count + 1e-5
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)
    

    def expand(self):
        action = random.choice(self.expandable_actions)
        self.expandable_actions.remove(action)

        child_state = self.state.copy()
        child_state = self.game.get_next_state(child_state, action, self.player)

        child = Node(self.game, self.args, child_state, 1 - self.player, self.turn_count + 1, self, action)
        self.children.append(child)
        return child
    
    def simulate(self):
        rollout_state = self.state.copy()
        rollout_player = self.player
        rollout_count = self.turn_count
        while True:
            # print(self.turn_count, rollout_count)
            is_terminal, is_draw, value = self.game.get_value_and_terminated(rollout_state, rollout_player, rollout_count)
            if is_terminal:
                return is_draw, value
            rollout_player = 1 - rollout_player

            valid_moves = self.game.get_valid_actions(rollout_state, rollout_player)
            action = random.choice(valid_moves)
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            rollout_count += 1

    def backpropagate(self, value, is_draw):
        self.visit_count += 1
        if is_draw:
            self.total_value += value * self.args['draw_discount']
        else:
            self.total_value += value

        if self.parent is not None:
            self.parent.backpropagate(value, is_draw)


class MCTS:
    def __init__(self, game: Quoridor, args) -> None:
        self.game = game
        self.args = args

    def search(self, state, player):
        root = Node(self.game, self.args, state, player, 0)

        for search in tqdm.tqdm(range(self.args['N'])):
            # print(search)
            node = root

            while node.is_fully_expanded():
                # print(node.expandable_actions, node.children)
                node = node.select()

            is_terminal, is_draw, value = self.game.get_value_and_terminated(node.state, 1 - node.player, node.turn_count)

            if not is_terminal:
                node = node.expand()
                is_draw, value = node.simulate()
            node.backpropagate(value, is_draw)
        
        action_probs = np.zeros(len(root.children))
        actions = [child.action_taken for child in root.children]
        data = [(child.visit_count, child.total_value) for child in root.children]
        for idx, child in enumerate(root.children):
            action_probs[idx] = child.total_value / child.visit_count
        print('Search nodes:', data)
        action_probs /= np.sum(action_probs)
        return actions, action_probs



In [54]:
quoridor = Quoridor(3, 2)
player = 0

args = {
    'C': 1.414,
    'N': 1000,
}

mcts = MCTS(quoridor, args)

state = quoridor.get_initial_state()

while True:
    print(state.board)
    print(state.left_wall)
    
    if player == 0:
        valid_actions = quoridor.get_valid_actions(state, player)
        print(valid_actions)
        cmd = input('Player 1: ')
        action = parse_cmd(cmd)

        if action not in valid_actions:
            print('Invalid action')
            continue
    else:
        mcts_actions, mcts_probs = mcts.search(state, player)
        print(mcts_actions, '\n', mcts_probs)
        action = mcts_actions[np.argmax(mcts_probs)]
        print(action)
    
    state = quoridor.get_next_state(state, action, player)
    is_terminal = quoridor.check_win(state, player)

    if is_terminal:
        print(state.board)
        print('Player {} wins'.format(player))
        break

    player = 1 - player


[[0 0 2 0 0]
 [0 1 0 1 0]
 [0 0 0 0 0]
 [0 1 0 1 0]
 [0 0 3 0 0]]
[2 2]
[(0, (0, 2)), (0, (0, 0)), (0, (1, 1)), (1, (0, 0, 0)), (1, (0, 0, 1)), (1, (0, 1, 0)), (1, (0, 1, 1)), (1, (1, 0, 0)), (1, (1, 0, 1)), (1, (1, 1, 0)), (1, (1, 1, 1))]


ValueError: Invalid action type

In [60]:
quoridor = Quoridor(5, 4)

args = {
    'C': 1.4,
    'N': 1000,
    'draw_discount': 0.2,
}

mcts = MCTS(quoridor, args)

state = quoridor.get_initial_state()

player = 1
while True:
    print(state.board)
    print(state.left_wall)
    
    mcts_actions, mcts_probs = mcts.search(state, player)
    print(mcts_actions, '\n', mcts_probs)
    action = mcts_actions[np.argmax(mcts_probs)]
    print(action)
    
    state = quoridor.get_next_state(state, action, player)
    is_terminal = quoridor.check_win(state, player)

    if is_terminal:
        print(state.board)
        print('Player {} wins'.format(player))
        break

    player = 1 - player


[[0 0 0 0 2 0 0 0 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 0 3 0 0 0 0]]
[4 4]


100%|██████████| 1000/1000 [01:46<00:00,  9.37it/s]


Search nodes: [(30, 13.685037478861007), (22, 7.576062369003545), (34, 16.817789455126917), (26, 10.520540701128935), (26, 10.69409848149786), (25, 9.63654401154401), (22, 7.755995888169802), (25, 9.960818538186958), (23, 8.549762827916187), (22, 7.683785005843829), (31, 14.877684090934864), (23, 8.558167096061831), (55, 35.52319347319347), (19, 5.461573150791417), (22, 7.7825752025752015), (33, 15.97149838465628), (25, 9.76646409146409), (37, 19.75156343656343), (29, 12.795791169614697), (22, 7.791280288339112), (25, 9.814380619380618), (22, 7.560464521270146), (23, 8.56716684622567), (24, 9.361794055617585), (45, 26.12067017949371), (35, 17.893704628704626), (35, 18.017715140037122), (19, 5.657623291740937), (45, 26.13482175681664), (33, 15.958054494525081), (31, 14.768154218502515), (30, 14.004596447694917), (32, 15.75924557113815), (28, 11.852041749100573), (22, 7.660679196104893)]
[(0, (3, 2)), (1, (1, 2, 2)), (1, (1, 2, 3)), (1, (0, 3, 0)), (1, (0, 2, 0)), (1, (1, 2, 1)), (0, (4,

100%|██████████| 1000/1000 [01:11<00:00, 13.92it/s]


Search nodes: [(26, 11.392092352092353), (28, 12.710514952883374), (32, 16.021867021867017), (23, 8.723864468864468), (20, 6.601733723792545), (33, 16.749996392496392), (37, 20.646581196581195), (31, 15.433791763791762), (39, 22.225045787545785), (54, 35.01458198664081), (26, 11.342409812409812), (40, 22.72717320261438), (23, 8.618989621489622), (24, 9.8018759018759), (29, 13.705664540138223), (44, 26.758798701298705), (25, 10.410581085581084), (32, 15.761848370927314), (33, 16.91602869352869), (28, 12.37400451887294), (32, 15.810414862914863), (40, 23.008185425685426), (34, 17.728354978354975), (49, 31.04671090605302), (37, 20.421255133755132), (41, 23.894501332001333), (33, 16.671002886002885), (28, 12.756311645870467), (24, 9.225122867328752), (24, 9.626559829059829), (31, 15.560768398268397)]
[(1, (0, 0, 0)), (1, (0, 2, 3)), (1, (1, 1, 3)), (1, (1, 1, 0)), (1, (1, 1, 2)), (1, (0, 1, 3)), (1, (1, 0, 0)), (1, (1, 2, 3)), (1, (0, 0, 3)), (1, (1, 3, 1)), (1, (0, 1, 2)), (0, (1, 2)), (1

...

Search nodes: [(280, 277.23333333333335), (271, 267.2476190476191), (198, 187.77214285714288), (251, 245.37)]
[(0, (3, 2)), (0, (2, 1)), (0, (0, 2)), (0, (2, 3))] 
 [0.25373442 0.25271821 0.24302892 0.25051845]
(0, (3, 2))
[[0 0 0 1 0 1 0 0 0]
 [0 1 0 1 0 1 1 1 1]
 [0 0 0 1 0 1 0 0 0]
 [1 1 1 1 0 1 1 1 1]
 [0 0 0 0 3 0 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 1 2 1 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0]]
[0 0]


100%|██████████| 1000/1000 [00:01<00:00, 942.32it/s]


Search nodes: [(267, 264.44166666666666), (222, 215.0759523809524), (261, 257.5444444444445), (250, 245.67000000000002)]
[(0, (1, 2)), (0, (4, 2)), (0, (2, 3)), (0, (2, 1))] 
 [0.25210019 0.2466002  0.25116911 0.25013051]
(0, (1, 2))
[[0 0 0 1 0 1 0 0 0]
 [0 1 0 1 0 1 1 1 1]
 [0 0 0 1 3 1 0 0 0]
 [1 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 1 2 1 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0]]
[0 0]


100%|██████████| 1000/1000 [00:00<00:00, 1685.73it/s]

Search nodes: [(428, 417.7666666666667), (572, 572)]
[(0, (2, 2)), (0, (4, 2))] 
 [0.49395026 0.50604974]
(0, (4, 2))
[[0 0 0 1 0 1 0 0 0]
 [0 1 0 1 0 1 1 1 1]
 [0 0 0 1 3 1 0 0 0]
 [1 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 0 0 1 2 1 0 0 0]]
Player 0 wins
